In [1]:
import findspark



findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark=SparkSession.builder.appName('EY-Prep').getOrCreate()
spark

In [3]:
df=spark.read.format("csv").option("header",True).option("inferSchema",True).\
              load("C:\\Users\\vivek\\Downloads\\stocks.small.csv")
df.show()

+-------------------+---------+----------+----------+----------+---------+---------+------+
|               date|     open|      high|       low|     close|   volume| adjclose|symbol|
+-------------------+---------+----------+----------+----------+---------+---------+------+
|2000-07-18 00:00:00| 144.8125|144.828125|  141.4375|     143.0|5.06836E7|50.155473|  INTC|
|2000-07-20 00:00:00| 32.93751|  34.25001|   32.8125|     33.75|3288300.0| 8.789734|   BEN|
|2000-07-24 00:00:00|    64.25| 67.312477| 64.187523|     64.75| 948800.0| 7.689567|   APH|
|2000-07-26 00:00:00|   21.875|    22.125|   20.9375|   20.9375|1464300.0| 15.61832|   SHW|
|2000-07-26 00:00:00|     42.0| 42.312481|    41.625|    41.875|1397600.0| 9.402721|   STJ|
|2000-07-31 00:00:00|33.937519| 33.999986|     33.75| 33.875011| 273400.0| 5.063753|   GGP|
|2000-08-07 00:00:00|41.375038|      43.0| 40.812481| 42.812481|2.43376E7| 4.869542|  SBUX|
|2000-08-09 00:00:00|55.562481|    55.625|      54.5|      55.0| 387600.0|10.495

In [12]:
#df.write.option("header",True).mode("overwrite").partitionBy(date_format(col('date'),'dd-MMM-yyyy')).csv('C:\\Users\\vivek\\Desktop\\Spark-Output\\Stock')

df1=df.withColumn("date",month(col("date")))
df1.write.option("header",True).mode("overwrite").partitionBy("date").csv('C:\\Users\\vivek\\Desktop\\Spark-Output\\Stock')

In [15]:
df.select(expr("month(date) as Month")).show()

+-----+
|Month|
+-----+
|    7|
|    7|
|    7|
|    7|
|    7|
|    7|
|    8|
|    8|
|    8|
|    8|
|    8|
|    9|
|    9|
|    9|
|   10|
|   10|
|   10|
|   10|
|   10|
|   10|
+-----+
only showing top 20 rows



In [17]:
address = [(1,"14851 Jeffrey Rd","DE"),
    (2,"43421 Margarita St","NY"),
    (3,"13111 Siemon Ave","CA")]
df=spark.createDataFrame(address,['id','address','country'])
df.show()

+---+------------------+-------+
| id|           address|country|
+---+------------------+-------+
|  1|  14851 Jeffrey Rd|     DE|
|  2|43421 Margarita St|     NY|
|  3|  13111 Siemon Ave|     CA|
+---+------------------+-------+



In [20]:
df.select(concat(df.address,lit("-"),df.country).alias("Full Address")).show(truncate=False)

+---------------------+
|Full Address         |
+---------------------+
|14851 Jeffrey Rd-DE  |
|43421 Margarita St-NY|
|13111 Siemon Ave-CA  |
+---------------------+



In [22]:
df.select(regexp_replace(df.address,'Rd','Road').alias('address')).show()
df.withColumn('address',regexp_replace(df.address,'Rd','Road')).show()

+------------------+
|           address|
+------------------+
|14851 Jeffrey Road|
|43421 Margarita St|
|  13111 Siemon Ave|
+------------------+

+---+------------------+-------+
| id|           address|country|
+---+------------------+-------+
|  1|14851 Jeffrey Road|     DE|
|  2|43421 Margarita St|     NY|
|  3|  13111 Siemon Ave|     CA|
+---+------------------+-------+



In [23]:
df.withColumn('country',when(col("country")=="DE","Denmark").when(col("country")=="NY","NewYork").when(col("country")=="CA","Canada").otherwise("NA")).show()

+---+------------------+-------+
| id|           address|country|
+---+------------------+-------+
|  1|  14851 Jeffrey Rd|Denmark|
|  2|43421 Margarita St|NewYork|
|  3|  13111 Siemon Ave| Canada|
+---+------------------+-------+



In [24]:
df_zip=spark.read.format("csv").option("header",True).option("inferSchema",True).load("C:\\Users\\vivek\\Downloads\\small_zipcode.csv")
df_zip.show()


+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|               null|   PR|     30100|
|  2|    704|    null|PASEO COSTA DEL SUR|   PR|      null|
|  3|    709|    null|       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|               null|   TX|      null|
+---+-------+--------+-------------------+-----+----------+



In [28]:
df_zip.fillna({'type':"NA",'city':"NA",'population':"1000"}).show()

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|                 NA|   PR|     30100|
|  2|    704|      NA|PASEO COSTA DEL SUR|   PR|      1000|
|  3|    709|      NA|       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|                 NA|   TX|      1000|
+---+-------+--------+-------------------+-----+----------+



In [29]:
df_zip.fillna("NA",subset=["type","city"]).show()

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|                 NA|   PR|     30100|
|  2|    704|      NA|PASEO COSTA DEL SUR|   PR|      null|
|  3|    709|      NA|       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|                 NA|   TX|      null|
+---+-------+--------+-------------------+-----+----------+



In [34]:
df_zip.dropna(subset=["population"]).show()

+---+-------+--------+-----------------+-----+----------+
| id|zipcode|    type|             city|state|population|
+---+-------+--------+-----------------+-----+----------+
|  1|    704|STANDARD|             null|   PR|     30100|
|  3|    709|    null|     BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|CINGULAR WIRELESS|   TX|     84000|
+---+-------+--------+-----------------+-----+----------+



In [65]:
stringcols=["type","city"]
intcols=["type","city"]
df_zip.show()
df_zip.select([count(when(col(c).isNull(),c)).alias(c) for c in df_zip.columns]).show()
df_zip.select([when(col(c).isNull(),"NA").otherwise(col(c)).alias(c) for c in stringcols]).show()

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|               null|   PR|     30100|
|  2|    704|    null|PASEO COSTA DEL SUR|   PR|      null|
|  3|    709|    null|       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|               null|   TX|      null|
+---+-------+--------+-------------------+-----+----------+

+---+-------+----+----+-----+----------+
| id|zipcode|type|city|state|population|
+---+-------+----+----+-----+----------+
|  0|      0|   2|   2|    0|         2|
+---+-------+----+----+-----+----------+

+--------+-------------------+
|    type|               city|
+--------+-------------------+
|STANDARD|                 NA|
|      NA|PASEO COSTA DEL SUR|
|      NA|       BDA SAN LUIS|
|  UNIQUE|  CINGULAR WIRELESS|
|STANDARD|                 NA|
+----

In [36]:
simpleData = (("James", "Sales", 3000), \
    ("Michael", "Sales", 4600),  \
    ("Robert", "Sales", 4100),   \
    ("Maria", "Finance", 3000),  \
    ("James", "Sales", 3000),    \
    ("Scott", "Finance", 3300),  \
    ("Jen", "Finance", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Sales", 4100) \
  )

columns= ["employee_name", "department", "salary"]
df = spark.createDataFrame(data = simpleData, schema = columns)

df.show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|         Jeff| Marketing|  3000|
|        Kumar| Marketing|  2000|
|         Saif|     Sales|  4100|
+-------------+----------+------+



In [48]:
from pyspark.sql.window import Window

df.withColumn("row_number",row_number().over(Window.partitionBy("department").orderBy(col("salary")))).show()

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|        Maria|   Finance|  3000|         1|
|        Scott|   Finance|  3300|         2|
|          Jen|   Finance|  3900|         3|
|        Kumar| Marketing|  2000|         1|
|         Jeff| Marketing|  3000|         2|
|        James|     Sales|  3000|         1|
|        James|     Sales|  3000|         2|
|       Robert|     Sales|  4100|         3|
|         Saif|     Sales|  4100|         4|
|      Michael|     Sales|  4600|         5|
+-------------+----------+------+----------+



In [58]:
from pyspark.sql.window import Window

df.withColumn("rank",dense_rank().over(Window.partitionBy("department").orderBy("salary"))).where(col("rank")==1).show()

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|        Maria|   Finance|  3000|   1|
|        Kumar| Marketing|  2000|   1|
|        James|     Sales|  3000|   1|
|        James|     Sales|  3000|   1|
+-------------+----------+------+----+



Transpose

In [72]:
df=spark.read.format("csv").option("header",True).option("inferSchema",True).load("C:\\Users\\vivek\\Downloads\\marks.csv")
df.show()

+-----+--------+-----+
| Name|Subjects|Marks|
+-----+--------+-----+
|Rohan|   Maths|   89|
|Rohan| Science|   88|
|Rohan|     Eng|   96|
| Puja|   Maths|   85|
| Puja| Science|   79|
| Puja|     Eng|   87|
+-----+--------+-----+



In [76]:
df.groupBy("Name").pivot("Subjects").sum("Marks").show()

+-----+---+-----+-------+
| Name|Eng|Maths|Science|
+-----+---+-----+-------+
| Puja| 87|   85|     79|
|Rohan| 96|   89|     88|
+-----+---+-----+-------+



WordCount

In [77]:
df=spark.read.text("C:\\Users\\vivek\\Downloads\\words.txt")
df

DataFrame[value: string]

In [91]:
df.select(explode(split(col("value")," ")).alias("words")).groupBy(col("words")).count().orderBy(col("count"))

+---------+-----+
|     word|count|
+---------+-----+
|      not|    1|
|terrorist|    1|
|     Name|    1|
|       Hi|    1|
|       is|    1|
|     khan|    1|
|      and|    1|
|       My|    1|
|        I|    2|
|        a|    2|
|       am|    2|
|   doctor|    1|
+---------+-----+



In [4]:
data=[("a","aa",1),("a","aa",2),("b","bb",3),("b","bb",4),("b","bb",5)]
df=spark.createDataFrame(data,["col1","col2","col3"])
df

DataFrame[col1: string, col2: string, col3: bigint]

In [7]:
df.groupBy("col1","col2").agg(collect_list("col3")).show()

+----+----+------------------+
|col1|col2|collect_list(col3)|
+----+----+------------------+
|   a|  aa|            [1, 2]|
|   b|  bb|         [3, 4, 5]|
+----+----+------------------+



In [8]:
#Pivot and UnPivot


In [9]:
#SCD-1

In [10]:
#SCD-2